In [8]:
# header files
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from PIL import Image
import cv2
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

print("Header files loaded!")

Header files loaded!


In [2]:
# hard-coded paths
model_path = "../epi_seg_unet.pth"
image_path = "../data/TCGA-23-1123_5.png"
output_path = "../results/mask_5.png"
device = "cpu"
patch_size = 200

In [ ]:
# functions
# apply segmentation model on a patch
def get_patch_segmentation(patch):
    # resize and normalize the patch
    patch = patch.transpose((2, 0, 1))
    patch = patch / 255
    
    # convert patch to tensor
    patch = torch.from_numpy(patch)
    patch = patch.unsqueeze(0)
    patch = patch.to(device, dtype=torch.float32)
    patch_output = net(patch)
    
    # convert the patch output to binary mask
    patch_output = torch.sigmoid(patch_output)
    patch_output = patch_output.detach().squeeze().cpu().numpy()
    patch_output = (patch_output>.5).astype(np.uint8)
    
    # return mask for the patch
    return patch_output


# update main output from patch segmentation output
def update_output(patch_output, patch_dims):
    for index1 in range(0, patch_output.shape[0]):
        for index2 in range(0, patch_output.shape[1]):
            output[patch_dims[0]+index1, patch_dims[1]+index2] = patch_output[index1, index2]

In [3]:
# read image and apply transforms
image = Image.open(image_path).convert('RGB')
image = image.resize((512, 512), Image.BICUBIC)
image = np.array(image).astype(np.uint8)
image = image.transpose((2, 0, 1))
image = image / 255
print(image.shape)

(3, 512, 512)


In [4]:
# load pretrained model
net = torch.load(model_path, map_location=device)
net.eval()
print(net)

UNet(
  (inc): DoubleConv(
    (double_conv): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
    )
  )
  (down1): Down(
    (maxpool_conv): Sequential(
      (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (1): DoubleConv(
        (double_conv): Sequential(
          (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
          (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (4): BatchNorm2d(128, eps=1e-05, moment

/Users/arpitdec5/opt/anaconda3/lib/python3.8/site-packages/torch/serialization.py:671: SourceChangeWarning: source code of class 'torch.nn.modules.container.Sequential' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/Users/arpitdec5/opt/anaconda3/lib/python3.8/site-packages/torch/serialization.py:671: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/Users/arpitdec5/opt/anaconda3/lib/python3.8/site-packages/torch/serialization.py:671: SourceChangeWarning: source code of class 'torch.nn.modules.batchnorm.BatchNorm2d' has changed. you can retrieve the origi

In [5]:
image = torch.from_numpy(image)
image = image.unsqueeze(0)
image = image.to(device, dtype=torch.float32)
image_output = net(image)

/Users/arpitdec5/opt/anaconda3/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ../c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
/Users/arpitdec5/opt/anaconda3/lib/python3.8/site-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  ../aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


In [ ]:
# generate masks for each patch of the tile
output = np.zeros((image.shape[0], image.shape[1]))
output = np.array(output).astype(np.uint8)
for index1 in range(0, image.shape[0], patch_size):
    for index2 in range(0, image.shape[1], patch_size):
        patch_output = get_patch_segmentation(image[index1:min(index1+patch_size, image.shape[0]), index2:min(index2+patch_size, image.shape[1])])
        update_output(patch_output, (index1, index2))
print(output.shape)

In [6]:
# save image
image_output = torch.sigmoid(image_output)
image_output = image_output.detach().squeeze().cpu().numpy()
image_output = (image_output>.5).astype(np.uint8)
image_output = Image.fromarray((image_output*255).astype(np.uint8))
image_output = image_output.resize((2000, 2000), Image.BICUBIC)
image_output.save(output_path)

In [19]:
mask = cv2.imread("../results/epithelium_stroma_masks/TCGA-23-1123_21000_12000_epi_stroma_mask.png", 0)
ret, mask = cv2.threshold(mask, 200, 255, cv2.THRESH_BINARY)
print(mask.shape)

(1000, 1000)


In [28]:
epi_value = float(sum([sum(i) for i in mask])) / float(mask.shape[0]*mask.shape[1])
print(epi_value)

150.4755


In [29]:
mask = 255. - mask
stroma_value = float(sum([sum(i) for i in mask])) / float(mask.shape[0]*mask.shape[1])
print(stroma_value)

104.5245


In [24]:
a = np.array([[255, 255, 255], [255, 255, 255], [255, 255, 255]])
print(sum(sum(255. - a)))

0.0


In [30]:
print(float(sum([sum(i) for i in mask])))

104524500.0


In [26]:
count1 = 0
count2 = 0
for index1 in range(0, 1000):
    for index2 in range(0, 1000):
        if mask[index1, index2] == 0:
            count1 += 1
        if mask[index1, index2] == 255:
            count2 += 1
print(count1)
print(count2)

409900
590100
